### Import package

In [120]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences

pd.set_option('future.no_silent_downcasting', True)

### Read in files

In [121]:
def read_protein_file(file_path):
    with open(file_path, "r") as file:
        # Read the entire contents of the file
        seq = ""
        str = ""

        sequences = []
        strings = []
        count = 0
        for line in file:
            if count < 8:
                pass
            count += 1

            line = line.strip()
            if line == "<>":
                seq = ""
                str = ""
            elif line == "<end>" or line=="end":
                sequences.append(seq)
                strings.append(str)
            else:
                letters = line.split(" ")
                if len(letters) == 2:
                    seq += letters[0]
                    str += letters[1] if letters[1] != "_" else "c"
        return sequences, strings


In [122]:
train_seq, train_str = read_protein_file("data/protein-secondary-structure.train")
test_seq, test_str = read_protein_file("data/protein-secondary-structure.test")

### Create sliding windows

In [123]:
def split_based_on_windows(data_seq, data_str, W=17):
    all_sequences = []
    all_strings = []
    for i in range(len(data_seq)):
        sequences = [data_seq[i][j:j+W] for j in range(0, len(data_seq[i]), W)]
        strings = [data_str[i][j:j+W] for j in range(0, len(data_str[i]), W)]

        all_sequences += sequences
        all_strings += strings

    train_df = pd.DataFrame({"sequence": all_sequences, "string": all_strings})
    return train_df


In [124]:
WINDOW_SIZE = 17

In [125]:
train_df = split_based_on_windows(train_seq, train_str, WINDOW_SIZE)
test_df = split_based_on_windows(test_seq, test_str, WINDOW_SIZE)

In [126]:
train_df.head()

,sequence,string
0,GVGTVPMTDYGNDVEYY,ccccccccccccccccc
1,GQVTIGTPGKSFNLNFD,cceecccccceecceee
2,TGSSNLWVGSVQCQASG,cccccceeccccccccc
3,CKGGRDKFNPSDGSTFK,ccccccccccccccccc
4,ATGYDASIGYGDGSASG,cccccccccccccccee


### One hot encode the data

In [127]:
PROTEIN_LETTERS = 'ACDEFGHIKLMNPQRSTVWXY'
SECONDARY_LETTERS = 'ceh'

In [128]:
def ohe_for_nn(sequences, strings):
    
    X_ohe = [[PROTEIN_LETTERS.index(letter) for letter in seq] for seq in sequences]
    y_ohe = [[SECONDARY_LETTERS.index(letter) for letter in string] for string in strings]

    max_length = max(len(seq) for seq in X_ohe)
    X_padded = pad_sequences(X_ohe, maxlen=max_length, padding='post')
    y_padded = pad_sequences(y_ohe, maxlen=max_length, padding='post')

    X = np.zeros((len(X_padded), max_length, len(PROTEIN_LETTERS)))
    y = np.zeros((len(y_padded), max_length, len(SECONDARY_LETTERS)))

    for i in range(len(X_padded)):
        for j, aa_index in enumerate(X_padded[i]):
            X[i, j, aa_index] = 1
        for j, structure_index in enumerate(y_padded[i]):
            y[i, j, structure_index] = 1

    return X, y

In [129]:
X_train, y_train = ohe_for_nn(train_df['sequence'], train_df['string'])
X_test, y_test = ohe_for_nn(test_df['sequence'], test_df['string'])

### Create the neural network

In [130]:
# Define the neural network architecture
model = Sequential([
    LSTM(128, input_shape=(WINDOW_SIZE, len(PROTEIN_LETTERS)), return_sequences=True),
    Dense(64, activation='tanh'),
    Dense(128, activation='linear'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    TimeDistributed(Dense(len(SECONDARY_LETTERS), activation='softmax')) # Apply Dense layer to each time step
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

predictions = model.predict(X_test)

Epoch 1/100
24/24 [==============================] - 3s 39ms/step - loss: 0.9925 - accuracy: 0.5752 - val_loss: 0.9587 - val_accuracy: 0.5580
Epoch 2/100
24/24 [==============================] - 0s 14ms/step - loss: 0.9198 - accuracy: 0.5767 - val_loss: 0.9237 - val_accuracy: 0.5590
Epoch 3/100
24/24 [==============================] - 0s 13ms/step - loss: 0.8820 - accuracy: 0.5819 - val_loss: 0.9205 - val_accuracy: 0.5744
Epoch 4/100
24/24 [==============================] - 0s 13ms/step - loss: 0.8640 - accuracy: 0.6003 - val_loss: 0.9055 - val_accuracy: 0.5782
Epoch 5/100
24/24 [==============================] - 0s 13ms/step - loss: 0.8562 - accuracy: 0.6093 - val_loss: 0.9060 - val_accuracy: 0.5722
Epoch 6/100
24/24 [==============================] - 0s 13ms/step - loss: 0.8516 - accuracy: 0.6063 - val_loss: 0.9303 - val_accuracy: 0.5797
Epoch 7/100
24/24 [==============================] - 0s 14ms/step - loss: 0.8520 - accuracy: 0.6115 - val_loss: 0.9113 - val_accuracy: 0.5772
Epoch 

### Convert the prediction back

In [131]:
def convert_pred_to_str(predictions):
    inv_structure_map = {0: 'c', 1:'e', 2:'h'}
    y_pred_classes = np.argmax(predictions, axis=-1)
    protein_array = np.vectorize(inv_structure_map.get)(y_pred_classes)

    array_strings = ["".join(map(str, row)) for row in protein_array]
    return array_strings

In [132]:
result = convert_pred_to_str(predictions)

In [133]:
test_df['prediction'] = result

In [134]:
test_df

,sequence,string,prediction
0,ENLKLGFLVKQPEEPWF,cccccceeeccccccch,ccccccccccccchccc
1,QTEWKFADKAGKDLGFE,hhhhhhhhhhhhhcccc,ccceeeechcccccccc
2,VIKIAVPDGEKTLNAID,cceeeccchhhhhhhhh,cececcccccceeehcc
3,SLAASGAKGFVICTPDP,hhhhccccccccccccc,cchhccccccccccccc
4,KLGSAIVAKARGYDMKV,cccchhhhhhhhhcccc,chcchheccccccchhh
...,...,...,...
185,VKLVSWYDNEFGYSERV,eeeecccchhhhhhhhh,cccccccccccceeeec
186,VDLMAHMASKE,hhhhhhhhhhc,cehhhhhhhcccccccc
187,SIPPEVKFNKPFVFLMI,cccceeecccceeeeee,ccccchccccccccccc
188,EQNTKSPLFMGKVVNPT,ecccceeeeeeeecccc,ccccccceeeccccccc
